# Exemple de chargement du dataset PlantVillage
Ce notebook montre comment importer et utiliser les fonctions de data_loader.py pour charger le dataset.

In [1]:
from IPython.display import display
from src.helpers.helpers import PROJECT_ROOT
from src.data_loader.data_loader import load_plantvillage_all, load_plantvillage_five_images

print(f"Racine du projet : {PROJECT_ROOT}")

Racine du projet : /Users/mackjb/repository/datasciencetest_reco_plante-1


In [2]:
from IPython.display import display
from src.helpers.helpers import PROJECT_ROOT
from src.data_loader.data_loader import generate_raw_data_plantvillage_segmented_all, generate_clean_and_resized, augment_minority_classes_cv2
import time
import hashlib
import pandas as pd
from pathlib import Path
from PIL import Image

In [ ]:
# # Génération du CSV complet raw_data_plantvillage_segmented_all.csv
# print("\nGénération du CSV complet raw_data_plantvillage_segmented_all.csv...")
# df_raw = generate_raw_data_plantvillage_segmented_all()
# print(df_raw.head())


Génération du CSV complet raw_data_plantvillage_segmented_all.csv...
Raw data CSV enregistré: /Users/mackjb/repository/datasciencetest_reco_plante-1/dataset/plantvillage/csv/raw_data_plantvillage_segmented_all.csv
                                            filepath  \
0  /Users/mackjb/repository/datasciencetest_reco_...   
1  /Users/mackjb/repository/datasciencetest_reco_...   
2  /Users/mackjb/repository/datasciencetest_reco_...   
3  /Users/mackjb/repository/datasciencetest_reco_...   
4  /Users/mackjb/repository/datasciencetest_reco_...   

                                            filename extension  file_size  \
0  091b0459-1a9a-4a60-b59f-3de2faedb6f8___RS_HL 2...       jpg      12253   
1  aba13626-8beb-474e-b609-300e7eb4e896___RS_HL 1...       jpg      11555   
2  ec906a90-7da2-420f-a3ab-5a67979868ee___RS_HL 2...       jpg      10976   
3  04cb725a-632d-44e0-9886-c84e2bbdd3da___RS_HL 2...       jpg      13027   
4  7e02bc12-a026-4aff-88b1-78dcd0a97fa1___RS_HL 1...       jpg 

In [3]:
# Génération du CSV clean et des images 256x256 PNG en une seule méthode
df_clean, output_dir = generate_clean_and_resized(force_refresh=True)
print(df_clean.head())
print(f"Clean CSV mis à jour et images PNG générées dans : {output_dir}")




Recalcul des données et des features (rafraîchissement forcé ou colonnes manquantes)...

Génération du CSV clean et des images 256x256 PNG en une seule passe...
                                            filepath  \
0  /Users/mackjb/repository/datasciencetest_reco_...   
1  /Users/mackjb/repository/datasciencetest_reco_...   
2  /Users/mackjb/repository/datasciencetest_reco_...   
3  /Users/mackjb/repository/datasciencetest_reco_...   
4  /Users/mackjb/repository/datasciencetest_reco_...   

                                            filename extension  file_size  \
0  091b0459-1a9a-4a60-b59f-3de2faedb6f8___RS_HL 2...       png      12253   
1  aba13626-8beb-474e-b609-300e7eb4e896___RS_HL 1...       png      11555   
2  ec906a90-7da2-420f-a3ab-5a67979868ee___RS_HL 2...       png      10976   
3  04cb725a-632d-44e0-9886-c84e2bbdd3da___RS_HL 2...       png      13027   
4  7e02bc12-a026-4aff-88b1-78dcd0a97fa1___RS_HL 1...       png      11704   

   width  height mode  num_channels  as

In [4]:

# Utilisation :
df_aug = augment_minority_classes_cv2(df_clean, image_col='filepath', label_col='species')

In [5]:
pd.set_option('display.max_colwidth', None)


# Marquer l’origine
df_clean['is_augmented'] = False
df_aug  ['is_augmented'] = True

# Concaténer tout pour garder la structure uniforme
df_full = pd.concat([df_clean, df_aug], ignore_index=True)

df_full.describe()

df_full.columns


Index(['filepath', 'filename', 'extension', 'file_size', 'width', 'height',
       'mode', 'num_channels', 'aspect_ratio', 'is_image_valid', 'is_black',
       'is_na', 'is_duplicate_after_first', 'phi1_distingue_large_vs_etroit',
       'phi2_distinction_elongation_forme', 'phi3_asymetrie_maladie',
       'phi4_symetrie_diagonale_forme', 'phi5_concavite_extremites',
       'phi6_decalage_torsion_maladie', 'phi7_asymetrie_complexe',
       'energie_basse_forme_feuille', 'energie_moyenne_texture_veines',
       'energie_haute_details_maladie', 'hog_moyenne_contours_forme',
       'hog_ecarttype_texture', 'pixel_ratio', 'leaf_segments', 'area',
       'perimeter', 'circularity', 'solidity', 'extent', 'eccentricity',
       'major_axis_length', 'minor_axis_length', 'compactness',
       'fractal_dimension', 'species', 'disease', 'is_augmented'],
      dtype='object')

In [7]:
from sklearn.model_selection import train_test_split



# 2. On split SÉRIEUSEMENT sur les seules lignes originales
orig = df_full[df_full['is_augmented']==False]
train_orig, test_orig = train_test_split(
    orig,
    test_size=0.2,
    stratify=orig['species'],
    random_state=42
)

# # 3. Construire le train final : 
# #    on ajoute **toutes** les augmentations au train, 
# #    et on garde le test uniquement sur les originaux
# train_final = pd.concat([train_orig, df_full[df_full['is_augmented']==True]], 
#                         ignore_index=True)
# test_final  = test_orig.copy()

# # 4. Séparer X/y
# drop_cols = [
#     'filepath','filename','extension','file_size',
#     'is_image_valid','is_black','is_na',
#     'is_duplicate_after_first','disease','is_augmented'
# ]
# X_train = train_final.drop(columns=drop_cols + ['species'])
# y_train = train_final['species']

# X_test  = test_final.drop(columns=drop_cols + ['species'])
# y_test  = test_final['species']

# # 5. Poursuivre avec scaling, entraînement, SHAP, etc.
# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler().fit(X_train)
# X_train_s = scaler.transform(X_train)
# X_test_s  = scaler.transform(X_test)

# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(
#     n_estimators=200,
#     class_weight='balanced',
#     random_state=42
# )
# clf.fit(X_train_s, y_train)

# # Vérification
# from sklearn.metrics import classification_report
# print(classification_report(y_test, clf.predict(X_test_s)))


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder().fit(y)
y_enc = le.transform(y)

print(y_enc)

[12 12 12 ... 10 10 10]


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc,
    test_size=0.2,
    stratify=y_enc,
    random_state=42
)